# <a name="ProjectHeader"></a> Проект А0
***
### *_Иследование реализация алгоритма "Угадать загаданное компьютером число за минимальное количество попыток"._*  


### Константы модуля

---

+ **`RANDOM_SEED`**  *Число используемое для инициализации генератора псевдослучайной последовательности. При значении `0` начальная инициализация отключена.* 
+ **`MAX_PASSES`**  *Количество запусков проверяемой функции.*
+ **`MAX_RUNDOM_SIZE`**  *Максимальное значение генерируемого числа.*
- **`_DBG`**  *Включение режима внутримодульной отладки(вывод промежуточных значени). По умолчанию отключено.* 





### Импортируемые модули

> *Для генерации случайного числа используется функция `random.randint()` из библиотеки `NumPy`*


In [1]:
import numpy as np

RANDOM_SEED = 5
"""См. np.random.seed(). Zerro - seed setup off"""
MAX_PASSES = 1000
"""Максимальное количество проходов проверки"""
MAX_RUNDOM_SIZE = 100
"""Максимальное загадываемое число"""
_DBG = 0
"""Вкл. отладочные принты"""

#!!! Блок инициализации ГПСЧ перенесен из функции score_game() в блок инициализации модуля
if RANDOM_SEED:
    np.random.seed(RANDOM_SEED) # фиксируем сид для воспроизводимости

### Функция многопроходной проверки. Сбор и вывод статистики.

---

~~~python
def score_game(random_predict) -> int
~~~
Функция многократной проверки алгоритма "угадывания", c накоплением резултатов и выводом статистики. Сохраняет количество попаданий на проход в локальном списке. Ее единственным параметром является функция реализующая алгортим "угадывания" числа в диапазоне `[1 - MAX_RANDOM_SIZE]` вызываемая данной функций с передаваемым параметром - внутренней функцией: 
#### Функция контроля
~~~python
    def verify_rnd_number(number:int=1) -> int
~~~  
> реализующая механизм контроля "угадывания" числа `number` и возвращающая `0` в случае "угадывания", `-1` если предложенное число меньше "загаданного" и `1` если предложенное число больше "загаданного". Фнутрення функция реализует механизм замыкания с локальной переменной rnd_num_ref,  являющейся ссылкой на список из одного значения - загаданного числа. *Логически скрывает "Загаданное" число от механиза "угадывания".*

В завершении `score_game()` выводит максимальное, минимальное и среднее количество затраченных попыток "угадывания" из общей выборки на 1000 проходов. Количество попыток "угадывания" за проход должна возвращать реализуемая функция, передаваемая параметром `random_predict`.


In [2]:
# для ограничения количества попыток
class GameOverError(OverflowError):
    pass

def score_game(random_predict) -> int:
    """
        За какое количество попыток в среднем из 1000 подходов угадывает наш алгоритм
    Args:
        random_predict (f: function([int])): функция угадывания
    Returns:
        int: среднее количество попыток
    """

    # список-ссылка на угадываемое число. (в замыкание)
    rnd_num_ref = [0]
    # счетчик попыток. (в замыкание)
    count_ref = [0]
    
    def verify_rnd_number(number:int=1) -> int:
        """
            Проверка числа на попадание
        Args:
            number (int, optional): Проверяемое число. Defaults to 1.

        Returns:
            int: 0 - если проверяемое число угадано
                 1 - если проверяемое число <меньше> загаданного
                -1 - если проверяемое число <больше> загаданного
        """
        nonlocal rnd_num_ref, count_ref
        
        if not type(number) is int:
            raise ValueError(f'value: [{number}] is not integer')
            #return None

        if count_ref[0] >= MAX_RUNDOM_SIZE:
            raise GameOverError('  Количество попыток "угадывания", увы, исчерпано :(')
        #TEST raise GameOverError('  Количество попыток "угадывания", увы, исчерпано :(')
                
        count_ref[0] += 1

        if rnd_num_ref[0] < number:
            return -1
        elif rnd_num_ref[0] > number:
            return 1
        else:
            return 0

    count_ls = [] # список для сохранения количества попыток

    #if RANDOM_SEED:
    #    np.random.seed(RANDOM_SEED) # фиксируем сид для воспроизводимости
    random_array = np.random.randint(1, MAX_RUNDOM_SIZE + 1, size=(MAX_PASSES)) # загадали список чисел

    for number in random_array:
        #TODO: change number -> rnd_num_ref[0] // Проверить
        rnd_num_ref[0] = number
        count_ref[0] = 0
        try:
        #
        # Запускаем наш "черный ящик" с функцией-ответчиком, фиксируем число "попыток"
        #
            random_predict(verify_rnd_number)
        #
        except Exception as e:
            print("\n\nОшибка выполнения \"random_predict()\" :")
            if isinstance(e, GameOverError):
                print(e)
                exit(1)    
            else:
                #raise (e.with_traceback)
                raise e
            
        count_ls.append(count_ref[0])

    score = int(np.mean(count_ls))   # находим среднее количество попыток
    min_hits = min(count_ls)         # мин
    max_hits = max(count_ls)         # макс

    print(f'\nВаш алгоритм угадывает число в среднем за: {score} попыток. Мин: {min_hits} , Макс: {max_hits}\n')
    return(score)


### Функция алгоритма "угадывания" случайного числа

---

#### Терминология
+ *"Загаданное" число*  - целое случайное число в заданном диапазоне, полученное от генератора случайных чисел библиотеки `NumPy` передаваемое функцией проверки в функцию контроля(через замыкание). Должно быть угадано за определенное количество попыток проверяемой функцией. *Используется псевдослучаная последовательность.  См.`random.seed()`*
+ *"Угадываемое" число* - целое число в заданном диапазоне передоваемое проверяемой функцией в функцию контроля, как возможное "Загаданное"

---

<a name="AlgDesc"></a> 
#### Алгоритм угадывания 

    Алгоритм угадывания основан на уменьшения диапазона в котором находится искомое "угадываемое" число. Случайно сгенерированное число передается функции контроля, которая возвращает значение "попадание". Или в случие не "попадания", возвращает больше или меньше "загаданное" число переданому. На основании последнего расчитывается новый диапазон для генерации ГСЧ "угадываемого" числа на следующей итерации.
---

Алгоритм "угадывания" числа реализован в модуле функцией:

~~~python
def random_predict(verify:callable) -> int
~~~

возвращающей количество попыток до успешного "угадывания" включительно. Входным параметром `verify` должна быть функция принимающая "угадываемое" число и возвращающая `0` в случии успеха. Если переданное число больше "угадываемого", то функция должна вернуть `-1`, если меньше, то `1`. 

> - *Пример реализации функции контроля в виде [lambda-функции](#ControlFunction "Функция контроля")*


In [6]:

def random_predict(verify:callable) -> int:
    """
        Рандомно угадываем число
        
    Args:
        verify : function(int)  функция проверяющая на попадание
            Принимает на вход угадывемое целое число.
            
            Дожна возвращать:
                0  - число угадано,
                1  - число больше предпологаемого,
                -1 - число меньше предпологаемого

    Returns:
        int: Число попыток
    """
    count = 0
    start_rnd_range = 1
    end_rnd_range = MAX_RUNDOM_SIZE

    # кол-во левых и правых "вилок" для отладки
    ch_rng_left  = 0
    ch_rng_right = 0

    while True:

        count += 1

        # случайное предполагаемое число
        predict_number = np.random.randint(start_rnd_range, end_rnd_range + 1) 

        control = verify(predict_number)
        if control == 0:
            if _DBG:  
                print(predict_number, ' - c:',count, ' <> ',ch_rng_left, ':',ch_rng_right)
            break
        elif control == 1:
            start_rnd_range = predict_number
            ch_rng_right += 1
        elif control == -1:
            end_rnd_range = predict_number
            ch_rng_left += 1
        else:
            raise ValueError("Control out of range. Mast be in [-1,0,1]")

    return count


<a name="ControlFunction"></a>
## Проверка работы функции "Угадывания"
[К описанию алгоритма](#AlgDesc "Описание алгоритма")

*с реализацией функции контроля, как лямбда-функция*

~~~python
_control_verify = lambda x : ... -> Literal[-1, 0, 1]
~~~

> *Для запуска проверки не забудте выполнить ячейки с импортом библиотеки `NumPy`, определением констант и реализацией 'random_predict()'  "Загаданное" значениние укажите в переменной _control_value*  

In [4]:
# проверка random_predict(control_function)
_control_value = MAX_RUNDOM_SIZE // 2
_control_verify = lambda x : 0 if x == _control_value else -1 if _control_value < x else 1 
print(f'Количество попыток: {random_predict(_control_verify)}')

Количество попыток: 12


## Вызов функции многопроходной проверки, накопления и вывода статистики

> *Для запуска проверки не забудте выполнить ячейки с импортом библиотеки `NumPy`, определением констант и ячеек с реализаций функций 'random_predict()' и 'score_game()'*

In [5]:
if __name__ == "__main__":
    score_game(random_predict)


Ваш алгоритм угадывает число в среднем за: 9 попыток. Мин: 1 , Макс: 31

